In [112]:
import pandas as pd
from openpyxl import load_workbook


In [113]:
df_medlemmar =  pd.read_excel(r"G:\Min enhet\Styrelsens dokument\Träning\Gradering.xlsx", sheet_name="Medlemmar")
df_medlemmar = df_medlemmar.drop(columns=['Skapad', 'Född', 'Startade', 'Tränat (år)' ])
df_medlemmar = df_medlemmar[df_medlemmar['Aktiv'] == 'Ja']
df_medlemmar['Aktiv'] = True
df_medlemmar['Ålder'] = pd.to_numeric(df_medlemmar['Ålder'], errors='coerce').fillna(0)

In [114]:
df_tekniker =  pd.read_excel(r"G:\Min enhet\Styrelsens dokument\Träning\Gradering.xlsx", sheet_name="Tekniker")
df_tekniker = df_tekniker[df_tekniker['Version'] == 'ÄJK 1.0']

In [115]:
df_graderingsregler =  pd.read_excel(r"G:\Min enhet\Styrelsens dokument\Träning\Gradering.xlsx", sheet_name="Graderingsregler")
df_graderingsregler = df_graderingsregler.drop(columns=['Ålder', 'Träningspass', '0'])
df_graderingsregler['Ålder2'] = pd.to_numeric(df_graderingsregler['Ålder2'], errors='coerce').astype(float)

In [116]:
age_rules = df_graderingsregler[['Ålder2', 'Dec']].sort_values('Ålder2')
tmp = pd.merge_asof( df_medlemmar.reset_index().sort_values('Ålder'), age_rules, left_on='Ålder', right_on='Ålder2', direction='backward')
df_medlemmar['Borde_ålder'] = tmp.sort_values('index').set_index('index')['Dec'].values

In [117]:
traningspass_rules = df_graderingsregler[['Akumelerat', 'Dec']].sort_values('Akumelerat')
tmp = pd.merge_asof( df_medlemmar.reset_index().sort_values('Närvaro'), traningspass_rules, left_on='Närvaro', right_on='Akumelerat', direction='backward')
df_medlemmar['Borde_närvaro'] = tmp.sort_values('index').set_index('index')['Dec'].values


In [118]:
df_medlemmar['Borde'] = df_medlemmar[['Borde_ålder', 'Borde_närvaro']].max(axis=1)
df_medlemmar = df_medlemmar.drop(columns=['Borde_ålder', 'Borde_närvaro'])

In [119]:
kyu_to_dec = df_graderingsregler.set_index('Kyu')['Dec']
kyu_to_dec

Kyu
6        6.00
6-1      5.50
5        5.00
5-1      4.75
5-2      4.50
5-3      4.25
4        4.00
4-1      3.75
4-2      3.50
4-3      3.25
3        3.00
3-1      2.75
3-2      2.50
3-3      2.25
2        2.00
2-1      1.75
2-2      1.50
2-3      1.25
1        1.00
1 dan     NaN
Name: Dec, dtype: float64

In [120]:


df_medlemmar['Grad_Dec'] = df_medlemmar['Grad'].map(kyu_to_dec)
df_medlemmar['Diff'] = df_medlemmar['Grad_Dec']-df_medlemmar['Borde']
df_medlemmar = df_medlemmar.sort_values('Diff')

df_medlemmar

,ID,Förnamn,Efternamn,Fullname,Personnummer,Adress,Postalcode,City,Ålder,Aktiv,Närvaro,Grad,Borde,Grad_Dec,Diff
67,TorbjörnJohansson,Torbjörn Johansson,NaN,Torbjörn Johansson,NaN,NaN,NaN,NaN,65.000000,True,101,1,3.50,1.00,-2.50
62,MickeKorths,Micke,Korths,Micke Korths,NaN,NaN,NaN,NaN,50.000000,True,156,1,3.00,1.00,-2.00
56,MilleKorths,Mille,Korths,Mille Korths,NaN,NaN,NaN,NaN,15.294444,True,114,2,3.50,2.00,-1.50
66,AndersSandblom,Anders,Sandblom,Anders Sandblom,NaN,NaN,NaN,NaN,65.000000,True,201,1,2.25,1.00,-1.25
10,AstridFjällgren,Astrid,Fjällgren,Astrid Fjällgren,NaN,NaN,NaN,NaN,6.230556,True,86,5,6.00,5.00,-1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,HarveyFundin,Harvey,Fundin,Harvey Fundin,NaN,NaN,NaN,NaN,14.177778,True,179,5-3,2.75,4.25,1.50
49,MilesFundin,Miles,Fundin,Miles Fundin,NaN,NaN,NaN,NaN,12.002778,True,173,5-3,2.75,4.25,1.50
63,RickardNisses-Gagnér,Rickard,Nisses-Gagnér,Rickard Nisses-Gagnér,NaN,NaN,NaN,NaN,52.000000,True,195,2 dan,2.50,NaN,NaN
65,DagSimmingskiöld,Dag Simmingskiöld,NaN,Dag Simmingskiöld,NaN,NaN,NaN,NaN,55.000000,True,10,6 dan,5.50,NaN,NaN


In [121]:
dec_to_kyu = df_graderingsregler.set_index('Dec')['Kyu']
df_medlemmar['Borde'] = df_medlemmar['Borde'].map(dec_to_kyu)
df_medlemmar = df_medlemmar.drop(columns=['Grad_Dec','Förnamn', 'Efternamn'])
df_medlemmar = df_medlemmar.rename(columns={'ID': 'email'})
df_medlemmar = df_medlemmar.rename(columns={'Närvaro': 'Attendance'})
df_medlemmar = df_medlemmar.rename(columns={'Grad': 'Grade'})
df_medlemmar = df_medlemmar.rename(columns={'Aktiv': 'Active'})
df_medlemmar = df_medlemmar.rename(columns={'Ålder': 'Age'})




df_medlemmar["password"] = "b252bdb8"
df_medlemmar["Club"] = "Ängelholms JK"


df_medlemmar

,email,Fullname,Personnummer,Adress,Postalcode,City,Age,Active,Attendance,Grade,Borde,Diff,password,Club
67,TorbjörnJohansson,Torbjörn Johansson,NaN,NaN,NaN,NaN,65.000000,True,101,1,4-2,-2.50,b252bdb8,Ängelholms JK
62,MickeKorths,Micke Korths,NaN,NaN,NaN,NaN,50.000000,True,156,1,3,-2.00,b252bdb8,Ängelholms JK
56,MilleKorths,Mille Korths,NaN,NaN,NaN,NaN,15.294444,True,114,2,4-2,-1.50,b252bdb8,Ängelholms JK
66,AndersSandblom,Anders Sandblom,NaN,NaN,NaN,NaN,65.000000,True,201,1,3-3,-1.25,b252bdb8,Ängelholms JK
10,AstridFjällgren,Astrid Fjällgren,NaN,NaN,NaN,NaN,6.230556,True,86,5,6,-1.00,b252bdb8,Ängelholms JK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,HarveyFundin,Harvey Fundin,NaN,NaN,NaN,NaN,14.177778,True,179,5-3,3-1,1.50,b252bdb8,Ängelholms JK
49,MilesFundin,Miles Fundin,NaN,NaN,NaN,NaN,12.002778,True,173,5-3,3-1,1.50,b252bdb8,Ängelholms JK
63,RickardNisses-Gagnér,Rickard Nisses-Gagnér,NaN,NaN,NaN,NaN,52.000000,True,195,2 dan,3-2,NaN,b252bdb8,Ängelholms JK
65,DagSimmingskiöld,Dag Simmingskiöld,NaN,NaN,NaN,NaN,55.000000,True,10,6 dan,6-1,NaN,b252bdb8,Ängelholms JK


In [122]:
df_medlemmar.columns = df_medlemmar.columns.str.lower()
df_medlemmar.to_json(r"..\judotech.web.club\source\data\medlemmar.json", orient="records", force_ascii=False, indent=2)
